## Multi-modal AI Tutor with UI, streaming, tool calls, and support for multiple model switching


### Define imports and model

In [29]:
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display

import base64
from io import BytesIO
import sqlite3

import gradio as gr
from PIL import Image
from openai import OpenAI

load_dotenv(override=True)

True

In [30]:
DEFAULT_MODEL = 'openai/gpt-4o-mini'
OPENROUTER_URL = "https://openrouter.ai/api/v1"
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")

In [31]:
openrouter = OpenAI(base_url=OPENROUTER_URL, api_key=OPENROUTER_API_KEY)

In [32]:
from prompts.ai_tutor import SYSTEM_PROMPT, USER_PROMPT

In [33]:
ERROR_TEMPLATE = "I'm sorry, I couldn't find an answer to your question.\nError: {error_message}\n"
ERROR_MSG_NO_RESPONSE = "No response from AI Tutor"
ERROR_MSG_NO_QUERY = "Query not found. Please ask a technical question!"

### Take user query and pass it to the AI Tutor

In [34]:
def get_response(question: str):
    """
    This function takes a user query and passes it to the AI Tutor.
    Args:
        question (str): user query
    Returns:
        response (str): response from the AI Tutor prompt
    """
    if not question:
        return ERROR_TEMPLATE.format(error_message=ERROR_MSG_NO_QUERY)
    response = openrouter.chat.completions.create(
        model=DEFAULT_MODEL,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": USER_PROMPT.format(question=question)}
        ]
    )
    return response.choices[0].message.content if response.choices[0] else \
        ERROR_TEMPLATE.format(error_message=ERROR_MSG_NO_RESPONSE)

### Move prompt options/list records to database (SQLite3)

In [ ]:
import json

AVAILABLE_MODELS = [
    "openai/gpt-4o-mini",
    "openai/gpt-4o",
    "anthropic/claude-3-5-sonnet",
    "google/gemini-2.0-flash-001",
    "meta-llama/llama-3.3-70b-instruct",
]

DB = "database/topics.db"

def get_topic_resources(topic: str) -> str:
    print(f"DATABASE TOOL CALLED: Getting resources for {topic}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute(
            "SELECT url FROM topic_resources WHERE ? LIKE '%' || topic || '%'",
            (topic.lower(),)
        )
        results = cursor.fetchall()
    if results:
        return f"Resources for {topic}:\n" + "\n".join(f"- {row[0]}" for row in results)
    return f"No specific resources found for '{topic}'. Try official docs or Stack Overflow."

get_topic_resources_fn = {
    "name": "get_topic_resources",
    "description": "Get curated learning resources and documentation links for a technical topic.",
    "parameters": {
        "type": "object",
        "properties": {
            "topic": {
                "type": "string",
                "description": "The technical topic to fetch resources for (e.g. Python, SQL, Machine Learning, LLMs)",
            },
        },
        "required": ["topic"],
        "additionalProperties": False
    }
}
tools = [{"type": "function", "function": get_topic_resources_fn}]

In [36]:
with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS topic_resources (
            topic TEXT,
            url   TEXT
        )
    """)
    cursor.execute("SELECT COUNT(*) FROM topic_resources")
    if cursor.fetchone()[0] == 0:
        cursor.executemany("INSERT INTO topic_resources (topic, url) VALUES (?, ?)", [
            ("python",           "https://docs.python.org/3/"),
            ("python",           "https://realpython.com/"),
            ("sql",              "https://www.w3schools.com/sql/"),
            ("sql",              "https://sqlzoo.net/"),
            ("machine learning", "https://scikit-learn.org/stable/"),
            ("machine learning", "https://www.kaggle.com/learn/"),
            ("deep learning",    "https://www.deeplearning.ai/"),
            ("deep learning",    "https://pytorch.org/tutorials/"),
            ("data science",     "https://pandas.pydata.org/docs/"),
            ("data science",     "https://numpy.org/doc/"),
            ("llm",              "https://huggingface.co/docs"),
            ("llm",              "https://platform.openai.com/docs"),
        ])
    conn.commit()
    print(f"DB ready: {DB}")

DB ready: topics.db


### Build image generation method with openrouter multimodal capability
Reference - https://openrouter.ai/docs/guides/overview/multimodal/image-generation

In [42]:
IMAGE_MODEL = "google/gemini-3-pro-image-preview"

def generate_diagram(question: str) -> Image.Image | None:
    try:
        prompt = (
            f"Create a clean, educational diagram or visual illustration to explain: '{question}'. "
            "Use a white background, clear labels, and a minimalist style. "
            "Show key concepts, relationships, or steps visually. No paragraphs of text."
        )
        response = openrouter.chat.completions.create(
            model=IMAGE_MODEL,
            messages=[{"role": "user", "content": prompt}],
            modalities=["image", "text"]
        )
        images = response.choices[0].message.model_dump().get('images', [])
        if not images:
            print("Diagram generation failed: no images in response", flush=True)
            return None
        image_data = base64.b64decode(images[0]['image_url']['url'].split(",", 1)[1])
        return Image.open(BytesIO(image_data))
    except Exception as e:
        print(f"Diagram generation failed: {e}", flush=True)
        return None

### Building chat, history, and handling tool calls

In [37]:
def handle_tool_calls(message):
    responses = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_topic_resources":
            arguments = json.loads(tool_call.function.arguments)
            topic = arguments.get('topic')
            print(f"TOOL CALLED: get_topic_resources({topic})", flush=True)
            responses.append({
                "role": "tool",
                "content": get_topic_resources(topic),
                "tool_call_id": tool_call.id
            })
    return responses


def put_message_in_chatbot(message, history):
    return "", history + [{"role": "user", "content": message}]


def chat(history, model):
    user_message = history[-1]["content"]
    messages = [{"role": "system", "content": SYSTEM_PROMPT}]
    for h in history[:-1]:
        messages.append({"role": h["role"], "content": h["content"]})
    messages.append({"role": "user", "content": USER_PROMPT.format(question=user_message)})

    response = openrouter.chat.completions.create(
        model=model or DEFAULT_MODEL,
        messages=messages,
        tools=tools
    )

    while response.choices[0].finish_reason == "tool_calls":
        tool_message = response.choices[0].message
        tool_responses = handle_tool_calls(tool_message)
        messages.append(tool_message)
        messages.extend(tool_responses)
        response = openrouter.chat.completions.create(
            model=model or DEFAULT_MODEL,
            messages=messages,
            tools=tools
        )

    history += [{"role": "assistant", "content": ""}]
    stream = openrouter.chat.completions.create(
        model=model or DEFAULT_MODEL,
        messages=messages,
        stream=True
    )
    for chunk in stream:
        history[-1]["content"] += chunk.choices[0].delta.content or ''
        yield history, None

    diagram = generate_diagram(user_message)
    yield history, diagram

### Add Gradio UI Layer

In [40]:
with gr.Blocks() as ui:
    gr.Markdown("# AI Tutor")
    model_selector = gr.Dropdown(
        choices=AVAILABLE_MODELS,
        value=DEFAULT_MODEL,
        label="Model"
    )
    with gr.Row():
        with gr.Column(scale=3):
            chatbot = gr.Chatbot(height=500, type="messages")
            message = gr.Textbox(label="Ask your question:")
        with gr.Column(scale=1):
            image_output = gr.Image(label="Visual Aid", interactive=False, height=500)

    message.submit(
        put_message_in_chatbot,
        inputs=[message, chatbot],
        outputs=[message, chatbot]
    ).then(
        chat,
        inputs=[chatbot, model_selector],
        outputs=[chatbot, image_output]
    )

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.
